In [1]:
import os
from datetime import datetime

import math
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageDraw

from scorefield.models.ddpm.denoising_diffusion import Unet
from scorefield.models.rrt.rrt import RRTStar
from scorefield.models.gaussian.heat_gaussdiffusion import GaussHeatDiffusion
from scorefield.utils.rl_utils import load_config
from scorefield.utils.utils import (
    gen_goals, gen_agents, overlay_goal, overlay_multiple, combine_objects, overlay_images,
    overlay_goal_agent, overlay_goals_agent, log_num_check,
    draw_obstacles_pil, convert_to_obstacle_masks,
    randgen_obstacle_masks, draw_obstacles_pixel,
    vector_field, clip_vectors
)
from scorefield.utils.diffusion_utils import bilinear_interpolate, bilinear_interpolate_samples

import matplotlib.pyplot as plt
import itertools
from typing import Optional
import shutil


# Args
config_dir = "./scorefield/configs/gaussian.yaml"
args = load_config(config_dir)
device = args['device']

bg = Image.open('assets/toy_exp/background0.png')
wastes = []
wastes.append(Image.open('assets/toy_exp/waste0.png'))
# wastes.append(Image.open('assets/toy_exp/waste4.png'))
# wastes.append(Image.open('assets/toy_exp/waste5.png'))

img_size = args['image_size']
noise_steps = args['noise_steps']
diffuse_t = args['diffuse_t']
sample_num = args['sample_num']

goal_bounds = args['goal_bounds']
agent_bounds = args['agent_bounds']
obstacle_pos = args['obstacles']

iterations = args['iterations']
train_lr = args['train_lr']
batch_size = args['batch_size']

class Unet2D(Unet):
    def __init__(
        self, 
        dim, 
        out_dim, 
        dim_mults=(1, 2, 4, 8),
    ):
        super().__init__(dim=dim, out_dim=out_dim, dim_mults=dim_mults)

    def forward(self, obs, t, x_t:Optional[torch.Tensor]=None):
        score_map = super().forward(obs, t)
        score = bilinear_interpolate_samples(score_map, x_t)    # output: (B,2)
        return score

    
model = Unet2D(
    dim=img_size,
    out_dim = 2,
    dim_mults = (1, 2, 4, 8),
).to(device)

optim = torch.optim.Adam(params=model.parameters(), lr=train_lr)

/home/mlcs_edf/mambaforge/envs/diffusion/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Train with random single goal

current_time = datetime.now().strftime('%b%d_%H-%M-%S')
log_dir = os.path.join('./runs/baselines/gaussian', current_time)
writer = SummaryWriter(log_dir)
shutil.copy('./scorefield/configs/gaussian.yaml', writer.log_dir)

assert batch_size % 2 == 0

# background = draw_obstacles_pil(bg, obstacle_pos)
# obstacle_masks = convert_to_obstacle_masks(batch_size, background[0].size, img_size, obstacle_pos)

for iters in tqdm(range(iterations)):
    obstacle_masks = randgen_obstacle_masks(batch_size, img_size)
    background = draw_obstacles_pixel(bg, obstacle_masks)
    multi_goals = gen_goals(goal_bounds, (2,batch_size//2), img_size, obstacles=obstacle_masks[:batch_size//2])
    multi_goal = (torch.rand(batch_size//2, 2, 2, device=device, dtype=torch.float32) * 0.2 - 0.1) * 0.05 + multi_goals
    single_goals = gen_goals(goal_bounds, batch_size//2, img_size, obstacles=obstacle_masks[batch_size//2:])
    single_goal = (torch.rand(batch_size//2, 1, 2, device=device, dtype=torch.float32) * 0.2 - 0.1) * 0.05 + single_goals
    obs1 = overlay_goal(background[:batch_size//2], img_size, wastes, multi_goal)
    obs2 = overlay_goal(background[batch_size//2:], img_size, wastes, single_goal)
    obs = torch.cat((obs1, obs2), dim=0)
    goal = torch.cat((multi_goal[:,0].unsqueeze(1), single_goal), dim=0)
    
    # goals = gen_goals(goal_bounds, batch_size, img_size, obstacles=obstacle_masks)
    # goal = (torch.rand(batch_size, 1, 2, device=device, dtype=torch.float32) * 0.2 - 0.1) * 0.1 + goals
    
    # obs = overlay_goal(background, img_size, wastes, goal)
    
    optim.zero_grad()
    
    losses = []
    target_scores = []
    pred_scores = []
    
    x0 = goal.clone().repeat_interleave(2, dim=1)
    for i in range(1, noise_steps+1):
        t = (torch.ones(1)*i).long().to(device)
            
        while True:
            x = x0 + torch.randn(batch_size, 2, 2, device=device, dtype=torch.float32) * math.sqrt(diffuse_t)
            if (x[...,:].abs() <=1.).all():
                break
        
        target_score = x0 - x
        target_score_normalized = target_score * math.sqrt(diffuse_t)
        score = model(obs, t, x)
        
        squared_diff = torch.sum((score - target_score_normalized)**2, dim=-1)
        
        losses.append(torch.mean(squared_diff))
        target_scores.append(target_score_normalized)
        pred_scores.append(score)
    
        x0 = x.clone()
    
    loss = sum(losses)/len(losses)
    loss.backward()
    optim.step()

    target_scores = torch.stack(target_scores).reshape(-1,2)
    pred_scores = torch.stack(pred_scores).reshape(-1,2)
    
    target_scores = torch.norm(target_scores.detach(), dim=-1)
    pred_scores = torch.norm(pred_scores.detach(), dim=-1)
    
    dotprod = torch.sum(target_scores.detach() * pred_scores.detach(), dim=-1)
    cosine_sim = dotprod / (torch.norm(target_scores.detach(), dim=-1) * torch.norm(pred_scores.detach(), dim=-1)+1e-8)
    
    writer.add_scalar("Loss/train", loss.item(), iters)
    writer.add_scalar("Norm/target_score", torch.mean(target_scores).item(), iters)
    writer.add_scalar("Norm/pred_score", torch.mean(pred_scores).item(), iters)
    writer.add_scalar("Similarity/dot_product", torch.mean(dotprod).item(), iters)
    writer.add_scalar("Similarity/cosine_similarity", torch.mean(cosine_sim).item(), iters)

    if iters % (iterations // 2) == 0:
        torch.save(model.state_dict(), os.path.join(writer.log_dir, f'model_params_{iters}.pt'))

torch.save(model.state_dict(), os.path.join(writer.log_dir, 'model_params.pt'))
            
writer.close()


  1%|          | 48/4000 [01:41<2:17:31,  2.09s/it]

In [11]:
len(pred_scores[0][0])

2